In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Data/training.csv')

In [22]:
df.head()

,INDEX,BEFORE_HEADLINE,BEFORE_BODY,AFTER_HEADLINE,AFTER_BODY
0,0,"Weather, LNG Exports Pace Near-, Long-Term Gas...",Changes in weather affect near-term gas prices...,Weather Volatility Keeps Gas Bulls Guessing on...,Domestic natural gas benchmark prices may be r...
1,1,Deutsche Bank Gains Pre-Market On DoJ Mortgage...,Deutsche Bank shares up 1% premarket after new...,DZ Bank Keeps Deutsche Bank at 'Sell' Despite ...,DZ Bank retains its sell rating on Deutsche Ba...
2,2,Select Sector Rally Suggests Bullish HK Undertone,"HK rebound today, which snaps 4-day losing str...",HK Short Sales Off 8.9% Friday;Bank Of China Tops,HK short-selling volume down 8.9% at HK$4.763 ...
3,3,Mexico City's F1 Grand Prix Drives Hotel Revpa...,Mexico's hotel revenue per available room rose...,Mexican Hotels' Revpar Surged on F1 Grand Prix...,Mexico's hotel revenue per available room rose...
4,4,Gas Injection Again Tops Estimate as Weather I...,Natural gas injections into storage continued ...,Gas Draw Tops Estimate as Weather Ups Demand Y...,The market produced the first withdrawals from...


In [23]:
exam = df.iloc[0]

In [24]:
exam[1]

'Weather, LNG Exports Pace Near-, Long-Term Gas Demand: Bull Case'

In [25]:
exam[2]

'Changes in weather affect near-term gas prices, and a mild summer this year may be followed by a warmer-than-average winter, which could pressure prices. Longer-term, the abundant supply of U.S. natural gas will require an additional demand outlet to soak up incremental output to sustain or raise prices. The likely emergence of exports of liquefied natural gas to Asia may satisfy that need. Industrial growth in the U.S. may raise natural gas consumption as well.'

In [26]:
exam[3]

'Weather Volatility Keeps Gas Bulls Guessing on Price Outlook'

In [27]:
exam[4]

'Domestic natural gas benchmark prices may be restrained in the near term given resilient output, in-line storage levels and expectations of less winter heating demand. Milder weather is forecast for the next eight to 14 days. While prices may show seasonal volatility, gas demand will remain restrained near-term if weather is subdued. Gas must compete for market share with coal and nuclear power. Near-term capacity constraints and weather will continue to affect winter basis differentials.'

일단 길이를 재보자. 그런데 뭔가 오류가 나는 항목이 있어 검사해보자.

In [35]:
bef_len = []
aft_len = []
for i in range(len(df)):
    try:
        bef_len.append(len(df['BEFORE_HEADLINE'][i].split()))
        aft_len.append(len(df['AFTER_HEADLINE'][i].split()))
    except:
        print(df['BEFORE_HEADLINE'][i], '\t', df['AFTER_HEADLINE'][i])

nan 	 NZX-50 Up 0.1%, But Volumes Remain Light -ForBarr
nan 	 NZX-50 Closes Down 0.8%; Telecom Weighs
NZX-50 +0.2% As Large Cap Stocks Rally 	 nan
nan 	 NZX-50 Edges Down; Market Awaits Greek Deal-ForBarr
NZX-50 Down 0.4% Dictated By Order Flow - Craigs 	 nan


nan때문이니 함수를 다음과 같이 짜자

In [36]:
def return_token_num(txt):
    try:
        tt = txt.split()
    except AttributeError:
        tt= ''
    return len(tt)

In [37]:
df['BEFORE_BODY_LEN'] = df['BEFORE_BODY'].apply(return_token_num)
df['AFTER_BODY_LEN'] = df['AFTER_BODY'].apply(return_token_num)

In [38]:
df['BEFORE_HEADLINE_LEN'] = df['BEFORE_HEADLINE'].apply(return_token_num)
df['AFTER_HEADLINE_LEN'] = df['AFTER_HEADLINE'].apply(return_token_num)

In [40]:
max(df['BEFORE_BODY_LEN']), max(df['AFTER_BODY_LEN']), max(df['BEFORE_HEADLINE_LEN']), max(df['AFTER_HEADLINE_LEN'])

(404, 404, 16, 17)

BERT 모델에 BEFORE_HEADLINE|BEFORE_BODY와 AFTER_HEADLINE|AFTER_BODY 를 랜덤하게 앞뒤로 넣어서 NSP 태스크로 해결해보자. shuffle = 0 이면 AFTER|BEFORE, 1이면 BEFORE|AFTER 로 하자.

In [64]:
df['Label'] = df['BEFORE_BODY_LEN'].apply(lambda x: random.randint(0,1))

In [102]:
X, X_BODY, Y, Y_BODY, Text, Text_BODY = [], [], [], [], [], []
for i in range(len(df)):
    if df['Label'][i] == 0:
        X.append(str(df['AFTER_HEADLINE'][i])+'|'+df['AFTER_BODY'][i])
        X_BODY.append(df['AFTER_BODY'][i])
        Y.append(str(df['BEFORE_HEADLINE'][i])+'|'+df['BEFORE_BODY'][i])
        Y_BODY.append(df['BEFORE_BODY'][i])
        Text.append('[CLS]'+str(df['AFTER_HEADLINE'][i])+'|'+df['AFTER_BODY'][i]+'[SEP]'+str(df['BEFORE_HEADLINE'][i])+'|'+df['BEFORE_BODY'][i])
        Text_BODY.append('[CLS]'+'|'+df['AFTER_BODY'][i]+'[SEP]'+'|'+df['BEFORE_BODY'][i])
    else:
        X.append(str(df['BEFORE_HEADLINE'][i])+'|'+df['BEFORE_BODY'][i])
        X_BODY.append(df['BEFORE_BODY'][i])
        Y.append(str(df['AFTER_HEADLINE'][i])+'|'+df['AFTER_BODY'][i])
        Y_BODY.append(df['AFTER_BODY'][i])
        Text.append('[CLS]'+str(df['BEFORE_HEADLINE'][i])+'|'+df['BEFORE_BODY'][i]+'[SEP]'+str(df['AFTER_HEADLINE'][i])+'|'+df['AFTER_BODY'][i])
        Text_BODY.append('[CLS]'+df['BEFORE_BODY'][i]+'|'+'[SEP]'+'|'+df['AFTER_BODY'][i])

In [75]:
X[0], X_BODY[0]

('Weather, LNG Exports Pace Near-, Long-Term Gas Demand: Bull Case|Changes in weather affect near-term gas prices, and a mild summer this year may be followed by a warmer-than-average winter, which could pressure prices. Longer-term, the abundant supply of U.S. natural gas will require an additional demand outlet to soak up incremental output to sustain or raise prices. The likely emergence of exports of liquefied natural gas to Asia may satisfy that need. Industrial growth in the U.S. may raise natural gas consumption as well.',
 'Changes in weather affect near-term gas prices, and a mild summer this year may be followed by a warmer-than-average winter, which could pressure prices. Longer-term, the abundant supply of U.S. natural gas will require an additional demand outlet to soak up incremental output to sustain or raise prices. The likely emergence of exports of liquefied natural gas to Asia may satisfy that need. Industrial growth in the U.S. may raise natural gas consumption as w

In [76]:
Y[0], Y_BODY[0]

('Weather Volatility Keeps Gas Bulls Guessing on Price Outlook|Domestic natural gas benchmark prices may be restrained in the near term given resilient output, in-line storage levels and expectations of less winter heating demand. Milder weather is forecast for the next eight to 14 days. While prices may show seasonal volatility, gas demand will remain restrained near-term if weather is subdued. Gas must compete for market share with coal and nuclear power. Near-term capacity constraints and weather will continue to affect winter basis differentials.',
 'Domestic natural gas benchmark prices may be restrained in the near term given resilient output, in-line storage levels and expectations of less winter heating demand. Milder weather is forecast for the next eight to 14 days. While prices may show seasonal volatility, gas demand will remain restrained near-term if weather is subdued. Gas must compete for market share with coal and nuclear power. Near-term capacity constraints and weath

In [100]:
Text[0]

'[CLS]Weather, LNG Exports Pace Near-, Long-Term Gas Demand: Bull Case|Changes in weather affect near-term gas prices, and a mild summer this year may be followed by a warmer-than-average winter, which could pressure prices. Longer-term, the abundant supply of U.S. natural gas will require an additional demand outlet to soak up incremental output to sustain or raise prices. The likely emergence of exports of liquefied natural gas to Asia may satisfy that need. Industrial growth in the U.S. may raise natural gas consumption as well.[SEP]Weather Volatility Keeps Gas Bulls Guessing on Price Outlook|Domestic natural gas benchmark prices may be restrained in the near term given resilient output, in-line storage levels and expectations of less winter heating demand. Milder weather is forecast for the next eight to 14 days. While prices may show seasonal volatility, gas demand will remain restrained near-term if weather is subdued. Gas must compete for market share with coal and nuclear power

In [72]:
df['Label'][0]

1

In [116]:
X[1], Y[1]

('DZ Bank Keeps Deutsche Bank at \'Sell\' Despite Settlement|DZ Bank retains its sell rating on Deutsche Bank, with at EUR15 price target, saying the settlement reached with U.S. and U.K. authorities in a money-laundering probe in Russia is "acceptable overall and substantially below expectations" of up to EUR1.5B ($1.6 billion). Despite this settlement, various legal matters have yet to be resolved, DZ says, with Deutsche Bank\'s capitalization remaining below peer-average. A tougher legal environment, Deutsche Bank\'s weak business development, and an unattractively high cost-revenue balance also continue to underpin DZ\'s negativity. Deutsche Bank shares are up 1.1% at EUR18.89, having slipped from early-day gains of 2.1%. ',
 "Deutsche Bank Gains Pre-Market On DoJ Mortgage Settlement|Deutsche Bank shares up 1% premarket after news late Tuesday that it completed --as expected-- a $7.2B settlement with the U.S. Justice Department over claims it misled investors on mortgage securities

dev셋을 20% 쓰자. 

In [103]:
df['X'] = X
df['X_BODY'] = X_BODY
df['Y'] = Y
df['Y_BODY'] = Y_BODY
df['Text'] = Text
df['Text_BODY'] = Text_BODY

In [117]:
df.head()

,INDEX,BEFORE_HEADLINE,BEFORE_BODY,AFTER_HEADLINE,AFTER_BODY,BEFORE_BODY_LEN,AFTER_BODY_LEN,BEFORE_HEADLINE_LEN,AFTER_HEADLINE_LEN,shuffle,Label,X,X_BODY,Y,Y_BODY,prob,class,Text,Text_BODY
0,0,"Weather, LNG Exports Pace Near-, Long-Term Gas...",Changes in weather affect near-term gas prices...,Weather Volatility Keeps Gas Bulls Guessing on...,Domestic natural gas benchmark prices may be r...,75,75,10,9,0,1,"Weather, LNG Exports Pace Near-, Long-Term Gas...",Changes in weather affect near-term gas prices...,Weather Volatility Keeps Gas Bulls Guessing on...,Domestic natural gas benchmark prices may be r...,0.715645,train,"[CLS]Weather, LNG Exports Pace Near-, Long-Ter...",[CLS]Changes in weather affect near-term gas p...
1,1,Deutsche Bank Gains Pre-Market On DoJ Mortgage...,Deutsche Bank shares up 1% premarket after new...,DZ Bank Keeps Deutsche Bank at 'Sell' Despite ...,DZ Bank retains its sell rating on Deutsche Ba...,120,98,8,9,0,0,DZ Bank Keeps Deutsche Bank at 'Sell' Despite ...,DZ Bank retains its sell rating on Deutsche Ba...,Deutsche Bank Gains Pre-Market On DoJ Mortgage...,Deutsche Bank shares up 1% premarket after new...,0.958877,dev,[CLS]DZ Bank Keeps Deutsche Bank at 'Sell' Des...,[CLS]|DZ Bank retains its sell rating on Deuts...
2,2,Select Sector Rally Suggests Bullish HK Undertone,"HK rebound today, which snaps 4-day losing str...",HK Short Sales Off 8.9% Friday;Bank Of China Tops,HK short-selling volume down 8.9% at HK$4.763 ...,90,57,7,9,1,1,Select Sector Rally Suggests Bullish HK Undert...,"HK rebound today, which snaps 4-day losing str...",HK Short Sales Off 8.9% Friday;Bank Of China T...,HK short-selling volume down 8.9% at HK$4.763 ...,0.990721,dev,[CLS]Select Sector Rally Suggests Bullish HK U...,"[CLS]HK rebound today, which snaps 4-day losin..."
3,3,Mexico City's F1 Grand Prix Drives Hotel Revpa...,Mexico's hotel revenue per available room rose...,Mexican Hotels' Revpar Surged on F1 Grand Prix...,Mexico's hotel revenue per available room rose...,83,87,10,10,1,1,Mexico City's F1 Grand Prix Drives Hotel Revpa...,Mexico's hotel revenue per available room rose...,Mexican Hotels' Revpar Surged on F1 Grand Prix...,Mexico's hotel revenue per available room rose...,0.350287,train,[CLS]Mexico City's F1 Grand Prix Drives Hotel ...,[CLS]Mexico's hotel revenue per available room...
4,4,Gas Injection Again Tops Estimate as Weather I...,Natural gas injections into storage continued ...,Gas Draw Tops Estimate as Weather Ups Demand Y...,The market produced the first withdrawals from...,83,91,10,11,0,0,Gas Draw Tops Estimate as Weather Ups Demand Y...,The market produced the first withdrawals from...,Gas Injection Again Tops Estimate as Weather I...,Natural gas injections into storage continued ...,0.849030,dev,[CLS]Gas Draw Tops Estimate as Weather Ups Dem...,[CLS]|The market produced the first withdrawal...


In [108]:
df_fin = df[['X', 'Y', 'Label']]

In [109]:
df_fin.to_csv('Data/training_preprocessed.csv')

이제 Text랑 훈련/검증셋으로 나누자.

In [111]:
df_fin['prob'] = [random.random() for _ in range(len(df_fin))]

def split_df(prob):
    if prob < 0.8: return 'train'
    else: return 'dev'
    
df_fin['class'] = df_fin['prob'].apply(split_df)

/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/inhwan/Python/Pytorch_ev/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [113]:
df_fin_train = df_fin[df_fin['class']=='train'][['X', 'Y', 'Label']]
df_fin_valid = df_fin[df_fin['class']=='dev'][['X', 'Y', 'Label']]

In [114]:
len(df_fin_train), len(df_fin_valid)

(160277, 39723)

In [115]:
df_fin_train.to_csv('Data/train.csv', index=False)
df_fin_valid.to_csv('Data/dev.csv', index=False)